# HarvestText中的关键词算法benchmark

In [1]:
import json
import pandas as pd
import numpy as np
import networkx as nx
from tqdm import tqdm
import jieba
from collections import defaultdict
from harvesttext import HarvestText

In [2]:
ht = HarvestText()

首先，选取的数据集是CLUE整理的CSL关键词预测数据集（[下载地址](https://github.com/CLUEbenchmark/CLUE#6-csl-%E8%AE%BA%E6%96%87%E5%85%B3%E9%94%AE%E8%AF%8D%E8%AF%86%E5%88%AB-keyword-recognition)）。需要先下载并放到本目录的`CSL关键词预测`文件夹下

在上面先在开发集上做一些基本的分析及调参。

In [9]:
data_dev = []
with open('CSL关键词预测/dev.json', encoding='utf-8') as f:
    for line in f:
        tmp = json.loads(line)
        data_dev.append((tmp['abst'], tmp['keyword']))
len(data_dev)

3000

一些基础的数据探索性分析（EDA）
- 每个文档的关键词个数
- 关键词的长度分布
- 考察分词`seg`的情况和不分词`nseg`的情况，有多少比例的关键词被覆盖。这决定了依赖分词和不依赖分词的算法所能达到的理论recall上限。

In [10]:
all_keywords = 0
recalls = {'seg':0, 'nseg':0}
kwd_cnt = defaultdict(int)
kwd_len_cnt = defaultdict(int)
for abst, kwds in data_dev:
    kwd_cnt[len(kwds)] += 1
    words = set(jieba.lcut(abst))
    all_keywords += len(kwds)
    recalls['seg'] += len(set(kwds) & words)
    recalls['nseg'] += sum(int(kwd in abst) for kwd in kwds)
    for kwd in kwds:
        kwd_len_cnt[len(kwd)] += 1


In [12]:
print(kwd_cnt)

defaultdict(<class 'int'>, {4: 1814, 3: 1128, 2: 58})


每篇文档的关键词数量在2-4之间

In [14]:
# 关键词长度的累积概率分布
pd.Series(kwd_len_cnt).sort_index().cumsum() / sum(kwd_len_cnt.values())

1     0.004277
2     0.260134
3     0.387970
4     0.702864
5     0.812756
6     0.904239
7     0.937151
8     0.956489
9     0.971551
10    0.980104
11    0.988100
12    0.991633
13    0.995258
14    0.995816
15    0.996281
16    0.997583
17    0.998791
18    0.999256
19    0.999442
20    0.999907
31    1.000000
dtype: float64

存在很长的关键词，以一个词而不是多词词组为单元的关键词算法无法处理这些情况，不过4个字以内也已经可以覆盖70%

In [11]:
for k in recalls:
    recalls[k] /= all_keywords
print(recalls)

{'seg': 0.3697471178876906, 'nseg': 0.7791000371885459}


上述情况说明，依赖jieba分词的算法在这个数据集上最多只能达到36.97%的recall，而其他从原文直接中抽取方法（新词发现，序列标注等）有可能达到77.91%。

下面的算法，因此在数值上不会有很好的表现，不过依旧可以为比较和调参提供一些参考。

给出一个关键词抽取的示例，包括`textrank`和HarvestText封装jieba并配置好参数和停用词的`jieba_tfidf`。

In [15]:
text, kwds = data_dev[10]
print(text)
print("真实关键词：", kwds)
print("jieba_tfidf 关键词(前5)：", ht.extract_keywords(text, 5, method="jieba_tfidf"))
print("textrank 关键词(前5)：", ht.extract_keywords(text, 5, method="textrank"))

随机噪声雷达通常利用时域相关完成脉冲压缩从而进行目标检测.该文根据压缩感知理论提出一种适用于噪声雷达目标检测的新算法,它用低维投影测量和信号重建取代了传统的相关操作和压缩处理,将大量运算转移到后期处理.该算法以噪声雷达所检测的目标空间分布满足稀疏性为前提；利用发射信号形成卷积矩阵,然后通过随机抽取卷积矩阵的行构建测量矩阵；并采用迭代收缩阈值算法实现目标信号重建.该文对算法作了详细的理论推导,形成完整的实现框架.仿真实验验证了算法的有效性,并分析了对处理结果影响较大的因素.该算法能够有效地重建目标,具有良好的运算效率.与时域相关法相比,大幅度减小了目标检测误差,有效抑制了输出旁瓣,并保持了信号的相位特性.
真实关键词：['目标', '相关', '矩阵']
jieba_tfidf 关键词(前5)：['算法', '矩阵', '检测', '目标', '信号']
textrank 关键词(前5)：['算法', '信号', '目标', '压缩', '矩阵']


每篇文章取前5个作为预测值，我们可以得到precision@5, recall@5, F1@5来评估算法的效果

In [ ]:
all_keywords = 0
pred_keywords = 0
recall_new_word = 0

In [19]:
topK = 5
ref_keywords, pred_keywords = 0, 0
acc_count = 0
for text, kwds in tqdm(data_dev):
    ref_keywords += len(kwds)
    pred_keywords += topK
    preds = ht.extract_keywords(text, topK, method="jieba_tfidf")
    acc_count += len(set(kwds) & set(preds))
prec = acc_count / pred_keywords
recall = acc_count / ref_keywords
f1 = 2*prec*recall/(prec+recall)
print(f"jieba Precison:{prec:.4f}, Recall:{recall:.4f}, F1:{f1:.4f}")

100%|██████████| 3000/3000 [00:29<00:00, 100.76it/s]
jieba Precison:{prec:.4f}, Recall:{recall:.4f}, F1:{f1:.4f}


In [20]:
print(f"jieba Precison:{prec:.4f}, Recall:{recall:.4f}, F1:{f1:.4f}")

jieba Precison:0.1060, Recall:0.1478, F1:0.1235


Textrank调参

In [21]:
from itertools import product

topK = 5
block_types = ["doc", "sent"]
window_sizes = [2, 3, 4]
if_weighted = [False, True]
for block_type, window, weighted in product(block_types, window_sizes, if_weighted):
    ref_keywords, pred_keywords = 0, 0
    acc_count = 0
    for text, kwds in tqdm(data_dev):
        ref_keywords += len(kwds)
        pred_keywords += topK
        preds = ht.extract_keywords(text, topK, method="textrank", block_type=block_type, window=window, weighted=weighted)
        acc_count += len(set(kwds) & set(preds))
    prec = acc_count / pred_keywords
    recall = acc_count / ref_keywords
    f1 = 2*prec*recall/(prec+recall)
    print(f"textrank[block: {block_type}, window:{window}, weighted:{weighted}] Precison:{prec:.4f}, Recall:{recall:.4f}, F1:{f1:.4f}")

100%|██████████| 3000/3000 [00:45<00:00, 66.11it/s]
textrank[block: doc, window:2, weighted:False] Precison:0.0942, Recall:0.1314, F1:0.1097
100%|██████████| 3000/3000 [00:46<00:00, 64.20it/s]
textrank[block: doc, window:2, weighted:True] Precison:0.0955, Recall:0.1332, F1:0.1113
100%|██████████| 3000/3000 [00:41<00:00, 71.53it/s]
textrank[block: doc, window:3, weighted:False] Precison:0.0948, Recall:0.1322, F1:0.1104
100%|██████████| 3000/3000 [00:41<00:00, 65.70it/s]
textrank[block: doc, window:3, weighted:True] Precison:0.0945, Recall:0.1318, F1:0.1101
100%|██████████| 3000/3000 [00:41<00:00, 72.11it/s]
textrank[block: doc, window:4, weighted:False] Precison:0.0944, Recall:0.1316, F1:0.1100
100%|██████████| 3000/3000 [00:41<00:00, 71.65it/s]
textrank[block: doc, window:4, weighted:True] Precison:0.0939, Recall:0.1309, F1:0.1093
100%|██████████| 3000/3000 [00:45<00:00, 66.37it/s]
textrank[block: sent, window:2, weighted:False] Precison:0.0931, Recall:0.1299, F1:0.1085
100%|██████████

textrank的最佳参数是 block: doc, window:2, weighted:True

precision和recall与jieba_tfidf还是有差距，可能是因为后者拥有从大量语料库中统计得到的idf数据能起到一定帮助

## 测试集benchmark

选取各个算法的最佳参数在测试集上获得最终表现

In [22]:
data_test = []
with open('CSL关键词预测/test.json', encoding='utf-8') as f:
    for line in f:
        tmp = json.loads(line)
        data_test.append((tmp['abst'], tmp['keyword']))
len(data_test)

3000

In [23]:
topK = 5
ref_keywords, pred_keywords = 0, 0
acc_count = 0
for text, kwds in tqdm(data_test):
    ref_keywords += len(kwds)
    pred_keywords += topK
    preds = ht.extract_keywords(text, topK, method="jieba_tfidf")
    acc_count += len(set(kwds) & set(preds))
prec = acc_count / pred_keywords
recall = acc_count / ref_keywords
f1 = 2*prec*recall/(prec+recall)
print(f"jieba Precison:{prec:.4f}, Recall:{recall:.4f}, F1:{f1:.4f}")

100%|██████████| 3000/3000 [00:30<00:00, 99.11it/s]
jieba Precison:0.1035, Recall:0.1453, F1:0.1209


In [24]:
topK = 5
ref_keywords, pred_keywords = 0, 0
acc_count = 0
for text, kwds in tqdm(data_test):
    ref_keywords += len(kwds)
    pred_keywords += topK
    preds = ht.extract_keywords(text, topK, method="textrank", block_size="doc", window=2, weighted=True)
    acc_count += len(set(kwds) & set(preds))
prec = acc_count / pred_keywords
recall = acc_count / ref_keywords
f1 = 2*prec*recall/(prec+recall)
print(f"textrank Precison:{prec:.4f}, Recall:{recall:.4f}, F1:{f1:.4f}")

100%|██████████| 3000/3000 [00:45<00:00, 65.51it/s]
textrank Precison:0.0955, Recall:0.1342, F1:0.1116


另，附上HarvestText与另一个流行的textrank的实现，[textrank4zh](https://github.com/letiantian/TextRank4ZH)的比较

In [26]:
from textrank4zh import TextRank4Keyword

def textrank4zh(text, topK, window=2):
    # same as used in ht
    allowPOS = {'n', 'ns', 'nr', 'nt', 'nz', 'vn', 'v', 'an', 'a', 'i'}
    tr4w = TextRank4Keyword(allow_speech_tags=allowPOS)
    tr4w.analyze(text=text, lower=True, window=window)
    return [item.word for item in tr4w.get_keywords(topK)]

text, kwds = data_dev[10]
print(text)
print("真实关键词：", kwds)
print("textrank4zh 关键词(前5)：", textrank4zh(text, 5))

随机噪声雷达通常利用时域相关完成脉冲压缩从而进行目标检测.该文根据压缩感知理论提出一种适用于噪声雷达目标检测的新算法,它用低维投影测量和信号重建取代了传统的相关操作和压缩处理,将大量运算转移到后期处理.该算法以噪声雷达所检测的目标空间分布满足稀疏性为前提；利用发射信号形成卷积矩阵,然后通过随机抽取卷积矩阵的行构建测量矩阵；并采用迭代收缩阈值算法实现目标信号重建.该文对算法作了详细的理论推导,形成完整的实现框架.仿真实验验证了算法的有效性,并分析了对处理结果影响较大的因素.该算法能够有效地重建目标,具有良好的运算效率.与时域相关法相比,大幅度减小了目标检测误差,有效抑制了输出旁瓣,并保持了信号的相位特性.
真实关键词：['目标', '相关', '矩阵']
textrank4zh 关键词(前5)：['算法', '信号', '目标', '压缩', '运算']


In [27]:
topK = 5
ref_keywords, pred_keywords = 0, 0
acc_count = 0
for text, kwds in tqdm(data_test):
    ref_keywords += len(kwds)
    pred_keywords += topK
    preds = textrank4zh(text, topK)
    acc_count += len(set(kwds) & set(preds))
prec = acc_count / pred_keywords
recall = acc_count / ref_keywords
f1 = 2*prec*recall/(prec+recall)
print(f"textrank4zh Precison:{prec:.4f}, Recall:{recall:.4f}, F1:{f1:.4f}")

100%|██████████| 3000/3000 [02:12<00:00, 24.17it/s]
textrank4zh Precison:0.0836, Recall:0.1174, F1:0.0977


HarvestText的textrank的实现在精度和速度上都有一定的优势。

总结各个算法在CSL数据及上的结果：

| 算法 | P@5 | R@5 | F@5 |
| --- | --- | --- | --- |
| textrank4zh | 0.0836 | 0.1174 | 0.0977 |
| ht_textrank | 0.0955 | 0.1342 | 0.1116 |
| ht_jieba_tfidf | **0.1035** | **0.1453** | **0.1209** |

综上，HarvestText的关键词抽取功能
- 把配置好参数的jieba_tfidf作为默认方法
- 使用自己的textrank实现而不是用流行的textrank4zh。